In [1]:
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter

Supervised NLP is used here to predict whether a sentence comes from _Alice in Wonderland_ by Lewis Carroll or _Persuasion_ by Jane Austen. Models used include Random Forests, Support Vector Machine, and Logistic Regression.

Word count features will be generated with a _Bag of Words_ approach: for each sentence, the number of times each word appears is counted.

In [2]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

In [3]:
# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [4]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [5]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)

sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


Count how often various words occur excluding stopwords and punctuation. We will work with lemmas (root words) rather than the raw text terms, and we'll only use the 2000 most common words for each text.

In [6]:
# Utility function to create a list of the 400 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(400)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [7]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head(2)

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000


,queen,visit,account,musgrove,opinion,gryphon,musgroves,short,colonel,scream,...,half,return,crofts,eat,young,queer,assure,likely,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll


## Trying out BoW

In [8]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.9761755485893417

Test set score: 0.8886278195488722


## BoW with Logistic Regression

In [9]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(3190, 622) (3190,)
Training set score: 0.9394984326018809

Test set score: 0.9130639097744361


# BoW with Gradient Boosting

In [10]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.8846394984326019

Test set score: 0.875


# Same model, new inputs

What if we feed the model a different novel by Jane Austen, like _Emma_?  Will it be able to distinguish Austen from Carroll with the same level of accuracy if we insert a different sample of Austen's writing?

First, we need to process _Emma_ the same way we processed the other data, and combine it with the Alice data:

In [11]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma)
print(emma[:100])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [12]:
# Parse our cleaned data.
emma_doc = nlp(emma)

In [13]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

# Emma is quite long, let's cut it down to the same length as Alice.
emma_sents = emma_sents[0:len(alice_sents)]

In [14]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

print('done')

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
done


Well look at that!  NLP approaches are generally effective on the same type of material as they were trained on. It looks like this model is actually able to differentiate multiple works by Austen from Alice in Wonderland.  Now the question is whether the model is very good at identifying Austen, or very good at identifying Alice in Wonderland, or both...

# Challenge 0:

Recall that the logistic regression model's best performance on the test set was 93%.  See what you can do to improve performance.  Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires.  Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 90%.  

# Challenge 1:
Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work.  This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

Record your work for each challenge in a notebook and submit it below.

## Extract data from another author
We'll start by extracting data from a new author, and creating the word count features and additional features.

The models will then be tested with all three authors at the same time.

Note that for processing time issues, the size of the bag of words was reduced from 2000 to 400 words.

In [15]:
# Extract and clean the Milton data
milton = gutenberg.raw('milton-paradise.txt')
milton = re.sub(r'VOLUME \w+', '', milton)
milton = re.sub(r'CHAPTER \w+', '', milton)
milton = text_cleaner(milton)
print(milton[:100])

# Parse our cleaned data.
milton_doc = nlp(milton)

# Group into sentences
milton_sents = [[sent, "Milton"] for sent in milton_doc.sents]

Book I Of Man's first disobedience, and the fruit Of that forbidden tree whose mortal taste Brought 


In [16]:
# Build a new Bag of Words data frame for Milton word counts.
# We'll use the same common words from Alice and Persuasion.
milton_sentences = pd.DataFrame(milton_sents)
milton_bow = bow_features(milton_sentences, common_words)

print('done')

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
done


In [17]:
# Concatenate the Milton bag of words with the word_counts data frame

wc_3_authors = pd.concat([word_counts, milton_bow], ignore_index=True)

wc_3_authors[wc_3_authors.isnull().any(axis=1)]

,queen,visit,account,musgrove,opinion,gryphon,musgroves,short,colonel,scream,...,half,return,crofts,eat,young,queer,assure,likely,text_sentence,text_source


In [18]:
# Create a feature to store sentence word counts
wc_3_authors['sent_wc'] = 0

# Process each row, counting the occurrence of words in each sentence
# and number of words per sentence
for i, sentence in enumerate(wc_3_authors.text_sentence):
  wc_3_authors.loc[i, 'sent_wc'] = len(sentence)
  for word in sentence:
    try:
      wc_3_authors.loc[i, word.pos_] +=1
    except KeyError:
      tag = word.pos_
      wc_3_authors[tag] = 0
      wc_3_authors.loc[i, word.pos_] +=1

# Add contextual information (length of previous and next sentences)
wc_3_authors['prev_sent_wc'] = wc_3_authors.sent_wc.shift(1)
wc_3_authors['next_sent_wc'] = wc_3_authors.sent_wc.shift(-1)

In [19]:
# Fill missing values resulting from the first and last
# rows when shifting columns
wc_3_authors.fillna(method='ffill', inplace=True)
wc_3_authors.fillna(method='bfill', inplace=True)

In [20]:
wc_3_authors.text_source.unique()

array(['Carroll', 'Austen', 'Milton'], dtype=object)

In [21]:
# Separate X and Y variables
selected_authors = ['Carroll', 'Austen']
carrol_austen_df = wc_3_authors[(wc_3_authors.text_source.isin(selected_authors))]

Y = carrol_austen_df['text_source']
X = np.array(carrol_austen_df.drop(['text_sentence','text_source'], 1))

# re-create the train and test data sets
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

## Random Forest 2nd attempt

In [22]:
rfc = ensemble.RandomForestClassifier()

train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.990282131661442

Test set score: 0.8740601503759399


Scores for the original Random Forest model:
- Training set score: 0.9805703541209652
- Test set score: 0.8801691729323309

## Logistic Regression 2nd attempt

In [23]:
lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(3190, 640) (3190,)
Training set score: 0.9410658307210031

Test set score: 0.9168233082706767


Scores from the original Linear Regression model:

- Training set score: 0.9388906298965841

- Test set score: 0.9064849624060151

# Gradient Boosting 2nd attempt

In [24]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.8915360501567398

Test set score: 0.875


Scores from the original Gradient Boosting model:
- Training set score: 0.8856157944218114
- Test set score: 0.8707706766917294

## Compare texts and identify Authors
Finally, we'll compare Emma, Alice, and Milton's 'Paradise' novel and see if the model is good at identifying the authors.

In [31]:
Y = wc_3_authors['text_source']
X = np.array(wc_3_authors.drop(['text_sentence','text_source'], 1))

# re-create the train and test data sets
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

# Model
lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))
lr_predicted = lr.predict(X_test)
pd.crosstab(y_test, lr_predicted)

Training set score: 0.9179550608122037

Test set score: 0.8685837971552257


col_0,Austen,Carroll,Milton
text_source,,,
Austen,1318,42,79
Carroll,111,514,52
Milton,125,16,977


# Conclusion - predicting a third author:

To improve performance, new "Parts Of Sentence Features" were created, including propostions, verbs, adverbs, adjectives, and other options from SpaCy. A feature with the current sentence word counts was also created, as well as previous and next sentence word counts.

SelectKbest was also tested, but the method only decreased model performance and was discarded.

Indeed, the model with additional features was able to predict the names of all three authors with relatively good accuracy. Also, the model's performance slightly increased with the new features. That said, there were three classes (or rather three authors) to predict, which adds complexity.

In the cell below, when the model is run to predict and score two authors, the accuracy increases by two percentage points, when compared with the original Linear Regression Model.

In [35]:
wc_milton_carrol = wc_3_authors[wc_3_authors.text_source != 'Austen']

Y = wc_milton_carrol['text_source']
X = np.array(wc_milton_carrol.drop(['text_sentence','text_source'], 1))

# re-create the train and test data sets
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

# Model
lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))
lr_predicted = lr.predict(X_test)
pd.crosstab(y_test, lr_predicted)

Training set score: 0.9669297256670425

Test set score: 0.9261971830985916


col_0,Carroll,Milton
text_source,,
Carroll,614,80
Milton,51,1030


Scores from the original Linear Regression model:

- Training set score: 0.9388906298965841

- Test set score: 0.9064849624060151